In [2]:
#Import Packages
import pandas as pd
import numpy as np
import os
import itertools
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datetime
import matplotlib
from herbie import Herbie
import pickle
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import pysolar.solar as solar
from geographiclib.geodesic import Geodesic
import xarray as xr
import pytz
import simplekml
import cartopy
import sys
sys.path.append('..')
import funcs.ac_funcs as ac
import funcs.plotting as plotting
pd.options.mode.chained_assignment = None

#Autoreload changed local modules
%load_ext autoreload
%autoreload 2

In [3]:
base_project_dir = '/Users/agmeyer4/LAIR_1/'

In [4]:
# Running this will load create the DEM handler 
dem_folder = os.path.join(base_project_dir,'Data/DEM/')
dem_fname = 'ASTGTM_NC.003_30m_aid0001_slcbig.nc'
dem_typeid = 'aster'
my_dem_handler = ac.DEM_handler(dem_folder,dem_fname,dem_typeid) #define the handler

In [15]:
#Instrument defs
instruments = {'WBB':{'lat':40.766,'lon':-111.847,'zasl':1470},
               'DBK':{'lat':40.538,'lon':-112.0696,'zasl':1584}
               ,
               'T1':{'lat':40.4697,'lon':-111.926,'zasl':1389},
               'T2':{'lat':40.549,'lon':-111.807,'zasl':1559},
               'T3':{'lat':40.640,'lon':-111.805,'zasl':1421},
               'T4':{'lat':40.700,'lon':-111.935,'zasl':1300},
               'T5':{'lat':40.661,'lon':-112.0736,'zasl':1502},
               'T6':{'lat':40.743,'lon':-112.177,'zasl':1288},
               'T7':{'lat':40.825,'lon':-112.005,'zasl': 1286},
               'T8':{'lat':40.855,'lon':-111.858,'zasl':1654}
               }

#Sources
landfills = {'slv_landfill':[40.746,-112.042],
             'jordan_landfill':[40.55862,-112.053],
             #'davis_landfill':[41.114,-111.931],
             #'weber_landfill':[41.218,-111.99],
             'bountiful':    [40.911,-111.917]}
ww_plants = {'central_valley_wwtp':[40.7036613,-111.9141398],
             'big_cottonwood_wwtp':[40.6187424,-111.7824328],
             'se_regional_wwtp':[40.5411975,-111.8191652],
             'south_valley_wwtp':[40.5033357,-111.9187493],
             'slc_wwtp':[40.8030915,-111.9295899],
             }
refineries = {'Chevron':        [40.825,-111.924],
              'Big West Oil':   [40.838,-111.920],
              'Marathon':       [40.794,-111.909],
              'Holly Refining': [40.887,-111.904],
              'Silver Eagle':   [40.868,-111.910]}

In [18]:
interval = '1T'

#Next define the datetime range
dt1_str = '2022-10-19 14:00:00'
dt2_str = '2022-10-20 04:00:00' #start datetime
timezone = 'UTC' #this should nearly always be UTC, but can identify if your datetime strings above are in a different timezone

#Now format the datetime and timezone into useful formats
dt1 = ac.dtstr_to_dttz(dt1_str,timezone)
dt2 = ac.dtstr_to_dttz(dt2_str,timezone)

#Define the heights above the instrument (in m) from which we would like to release particles
z_ail_list = [0,10000]

In [19]:
slant_dfs = {}
for name,details in instruments.items():
    gsh = ac.ground_slant_handler(details['lat'],details['lon'],details['zasl'],z_ail_list)
    slant_df = gsh.run_slant_at_intervals(dt1,dt2,my_dem_handler,interval=interval)
    slant_df = slant_df.dropna()
    slant_dfs[name] = slant_df

Adding receptor lat/lons along the slant column


/Users/agmeyer4/opt/anaconda3/envs/atmos_column/lib/python3.11/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \


Adding surface height and receptor elevation above ground level
adding surface and agl heights
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is 

/Users/agmeyer4/opt/anaconda3/envs/atmos_column/lib/python3.11/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \


Adding surface height and receptor elevation above ground level
adding surface and agl heights
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is 

...

Adding surface height and receptor elevation above ground level
adding surface and agl heights
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is 

/Users/agmeyer4/opt/anaconda3/envs/atmos_column/lib/python3.11/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \


Adding surface height and receptor elevation above ground level
adding surface and agl heights
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is 

/Users/agmeyer4/opt/anaconda3/envs/atmos_column/lib/python3.11/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \


Adding surface height and receptor elevation above ground level
adding surface and agl heights
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is outside the DEMs range
point is 

In [20]:
name = 'column_network'

kml_folder = os.path.join(base_project_dir,'Figures','KML')
kml_name = f'{name}.kml' #storage
icon_size = 0.6 #how big we want the receptor points to be 

kml = simplekml.Kml() #initialize the kml file

#Add landfills
fol = kml.newfolder(name='landfills')
for name,loc in landfills.items():
    pnt = fol.newpoint(name=name, coords=[(loc[1],loc[0],100)])
    pnt.style.iconstyle.color = simplekml.Color.cyan
    pnt.style.iconstyle.icon.href = 'https://maps.google.com/mapfiles/kml/pal2/icon18.png'
    pnt.style.iconstyle.scale = 0.5
    pnt.style.labelstyle.scale = 0
    pnt.altitudemode='relativeToGround'

#Add wwtps
fol = kml.newfolder(name='wwtps')
for name,loc in ww_plants.items():
    pnt = fol.newpoint(name=name, coords=[(loc[1],loc[0],100)])
    pnt.style.iconstyle.color = simplekml.Color.orange
    pnt.style.iconstyle.icon.href = 'https://maps.google.com/mapfiles/kml/pal2/icon18.png'
    pnt.style.iconstyle.scale = 0.2
    pnt.style.labelstyle.scale = 0
    pnt.altitudemode='relativeToGround'

#Add refineries
fol = kml.newfolder(name='refineries')
for name,loc in refineries.items():
    pnt = fol.newpoint(name=name, coords=[(loc[1],loc[0],100)])
    pnt.style.iconstyle.color = simplekml.Color.green
    pnt.style.iconstyle.icon.href = 'https://maps.google.com/mapfiles/kml/pal2/icon18.png'
    pnt.style.iconstyle.scale = 0.5
    pnt.altitudemode='relativeToGround'
    pnt.style.labelstyle.scale = 0

#Add the slant columns
for inst_id,slant_df in slant_dfs.items():
    fol = kml.newfolder(name=inst_id)
    for dt,newdf in slant_df.groupby(level=0): #groupby the first index -- datetime
        time_str = dt.strftime('%H:%M %Z') #grab the datetime string for labeling
        plotdf = newdf.droplevel(0).reset_index() #get the new dataframe at that datetime level and reset the index so z_asl is a column not an index

        slant_id = time_str #the slant ID is best associated with the time string 
        
        #the linestring is just a path between the first and last points of the column receptor, to visualize the column
        ls = fol.newlinestring(name=f'{slant_id}_ls',coords=[(plotdf.iloc[0]['receptor_lon'],plotdf.iloc[0]['receptor_lat'],plotdf.iloc[0]['receptor_zasl'])
                                                            ,(plotdf.iloc[-1]['receptor_lon'],plotdf.iloc[-1]['receptor_lat'],plotdf.iloc[-1]['receptor_zasl'])],
                                                            altitudemode='absolute')
        ls.timespan.begin = dt.strftime('%Y-%m-%dT%H:%M:%S')
        end = dt + pd.to_timedelta(interval)
        ls.timespan.end = end.strftime('%Y-%m-%dT%H:%M:%S')

kml.save(os.path.join(kml_folder,kml_name))